# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [33]:
import requests
import pyowm
import json

api_key = '8544888c42a4b56a401863d10dc5d872'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [9]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 292.37,
  'feels_like': 292.59,
  'temp_min': 291.49,
  'temp_max': 293.22,
  'pressure': 995,
  'humidity': 86,
  'sea_level': 995,
  'grnd_level': 991},
 'visibility': 10000,
 'wind': {'speed': 8.23, 'deg': 220},
 'clouds': {'all': 40},
 'dt': 1756892262,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1756876593,
  'sunset': 1756925015},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [10]:
from pyowm import OWM

In [13]:
from pyowm.owm import OWM
owm = OWM('ef95e5282de8444d2c5847ef8647d575')
mgr = owm.weather_manager()

obs = mgr.weather_at_place("London,GB")   # city, country
w = obs.weather

print("What's it like?", w.detailed_status)              # e.g., "light rain"
print("How warm?", w.temperature('celsius')["temp"], "°C")
print("How wet?", w.humidity, "% humidity")

What's it like? broken clouds
How warm? 18.2 °C
How wet? 88 % humidity


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [18]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 293.7°C, Humidity: 80%, Wind Speed: 6.69 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [37]:
owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place("London,GB")
w = observation.weather

# choose units: 'celsius', 'fahrenheit', or 'kelvin'
temperature = w.temperature('celsius')['temp']
humidity = w.humidity
wind_speed = w.wind().get('speed', 0)

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 20.53°C, Humidity: 79%, Wind Speed: 7.6 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [31]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [30]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-09-03 15:00:00, Temp: 294.52°C
Time: 2025-09-03 18:00:00, Temp: 295.47°C
Time: 2025-09-03 21:00:00, Temp: 296.3°C
Time: 2025-09-04 00:00:00, Temp: 293.85°C
Time: 2025-09-04 03:00:00, Temp: 291.49°C
Time: 2025-09-04 06:00:00, Temp: 290.27°C
Time: 2025-09-04 09:00:00, Temp: 292.23°C
Time: 2025-09-04 12:00:00, Temp: 293.04°C
Time: 2025-09-04 15:00:00, Temp: 296.63°C
Time: 2025-09-04 18:00:00, Temp: 298.16°C
Time: 2025-09-04 21:00:00, Temp: 297.04°C
Time: 2025-09-05 00:00:00, Temp: 295.74°C
Time: 2025-09-05 03:00:00, Temp: 295.54°C
Time: 2025-09-05 06:00:00, Temp: 293.38°C
Time: 2025-09-05 09:00:00, Temp: 293.57°C
Time: 2025-09-05 12:00:00, Temp: 293.96°C
Time: 2025-09-05 15:00:00, Temp: 297.42°C
Time: 2025-09-05 18:00:00, Temp: 299.99°C
Time: 2025-09-05 21:00:00, Temp: 300.87°C
Time: 2025-09-06 00:00:00, Temp: 299.19°C
Time: 2025-09-06 03:00:00, Temp: 297.69°C
Time: 2025-09-06 06:00:00, Temp: 297.4°C
Time: 2025-09-06 09:00:00, Temp: 296.43°C
Time: 2025-09-06 12:00:00, Temp: 296

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [47]:
owm = OWM("a14cdbc91a8575c20f4ddfdfd8e2bc66")
mgr = owm.weather_manager()
#daily_forecast = mgr.forecast_at_place('Berlin,DE', 'daily').forecast
three_h_forecast = mgr.forecast_at_place('Berlin,DE', '3h').forecast

TypeError: WeatherManager.forecast_at_place() missing 1 required positional argument: 'interval'

In [51]:
from pyowm.owm import OWM
owm = OWM('a14cdbc91a8575c20f4ddfdfd8e2bc66')
mgr = owm.weather_manager()
#daily_forecast = mgr.forecast_at_place('Berlin,DE', 'daily').forecast
three_h_forecast = mgr.forecast_at_place('Berlin,DE', '3h').forecast


In [54]:
three_h_forecast

<pyowm.weatherapi30.forecast.Forecast - reception_time=2025-09-03 14:46:54+00:00, interval=3h>

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [55]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 293.82°C, Fahrenheit: 560.876°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [56]:
owm = OWM('a14cdbc91a8575c20f4ddfdfd8e2bc66')
mgr = owm.weather_manager()

w = mgr.weather_at_place("London,GB").weather   # Observation → Weather

# pyowm converts units for you:
temp_c = w.temperature('celsius')['temp']       # e.g. 17.3
temp_f = w.temperature('fahrenheit')['temp']    # e.g. 63.1

print(f"Temperature in Celsius: {temp_c}°C, Fahrenheit: {temp_f}°F")

Temperature in Celsius: 20.69°C, Fahrenheit: 69.24°F
